<a href="https://colab.research.google.com/github/corrieann/lotsofbranches/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports

!pip install keras_tuner -q
import tensorflow as tf
import keras_tuner
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import sys
import time

In [ ]:
# Mounting google drive for training in google co lab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')

data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
AUTOTUNE=tf.data.AUTOTUNE

start = time.time()
dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(      # Testing dataset
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

end = time.time()
print("time elapsed = ", (end - start))

Found 8582 files belonging to 4 classes.
Using 6866 files for training.
Found 8582 files belonging to 4 classes.
Using 1716 files for validation.
Found 1705 files belonging to 4 classes.
time elapsed =  32.22160363197327


In [ ]:
# catch and prefetch for shorter training time

# AUTOTUNE=tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# downloading the weights oft the base model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = False,
    weights = "imagenet"
)

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
#saving the downloaded base_model

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model.save(saving_path)

In [ ]:
#Model

model_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp):

inputs = tf.keras.Input(shape=(299, 299, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)

# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005670323757088328),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

#  return model

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Transfer",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val)

# tuner.results_summary()


model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=15,
    validation_data = dataset_path_val)
    #verbose = 1)


model.evaluate(dataset_path_test, batch_size=5, verbose=1)               # evaluating using the test dataset

model.summary()

base_model.trainable = True                                           # switching the base_model to trainable

for layer in base_model.layers:                                       # Swtching all batchnormalization layer of the base_model to non-trainable to not lose weights
  if isinstance(layer, tf.keras.layers.BatchNormalization):
    layer.trainable = False

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001),    # compiling the model with a trainable base_model
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

# model.fit(                                                            # fitting for another 10 epochs starting where the non-trainable ended
#     dataset_path,
#     epochs=30,
#     initial_epoch=15,
#     validation_data = dataset_path_val)
    #verbose = 1)

model.evaluate(dataset_path_test, batch_size=5, verbose=1)

model.summary()

# saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedModelWithFT/')

# tf.keras.models.save_model(model,                                     # saving the fully trained model
#                            saving_path,
#                            overwrite=True,
#                            save_format='tf'
#                            )

Epoch 1/15
1374/1374 [==============================] - 2010s 1s/step - loss: 0.5756 - accuracy: 0.7830 - val_loss: 0.4003 - val_accuracy: 0.8625
Epoch 2/15
1374/1374 [==============================] - 486s 354ms/step - loss: 0.3808 - accuracy: 0.8602 - val_loss: 0.3462 - val_accuracy: 0.8759
Epoch 3/15
1374/1374 [==============================] - 488s 355ms/step - loss: 0.3311 - accuracy: 0.8793 - val_loss: 0.2980 - val_accuracy: 0.8887
Epoch 4/15
1374/1374 [==============================] - 493s 359ms/step - loss: 0.3019 - accuracy: 0.8865 - val_loss: 0.2656 - val_accuracy: 0.9091
Epoch 5/15
1374/1374 [==============================] - 493s 359ms/step - loss: 0.2756 - accuracy: 0.9002 - val_loss: 0.2547 - val_accuracy: 0.9108
Epoch 6/15
1374/1374 [==============================] - 489s 356ms/step - loss: 0.2570 - accuracy: 0.9049 - val_loss: 0.2322 - val_accuracy: 0.9184
Epoch 7/15
1374/1374 [==============================] - 489s 356ms/step - loss: 0.2499 - accuracy: 0.9093 - val_lo